In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# df = pd.read_excel('drive/My Drive/graduate/data/Canon_MD.xlsx', sheet_name=None)

Mounted at /content/drive


In [3]:
# nltk stopwords merging with another stopwords list from https://github.com/ahmetax/trstop/blob/master/dosyalar/turkce-stop-words
nltk.download('stopwords')
stops_tr = stopwords.words("turkish")

lines = []
with open("drive/My Drive/graduate/data/extended_stopwords.txt",encoding="utf-8") as f:
  lines = f.readlines()

f.close()

lines = [item.replace('\n','') for item in lines]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
print(stops_tr)
print('stops_tr length: ', len(stops_tr))

print(lines)
print('lines length: ', len(lines))

stops_tr.extend(lines)
print(stops_tr)
print('extended stops_tr length:', len(stops_tr))

stops_tr = np.array(stops_tr)
stops_tr = np.unique(stops_tr)
print('final unique stopwords: ', len(stops_tr))

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']
stops_tr length:  53
['a', 'acaba', 'altı', 'altmış', 'ama', 'ancak', 'arada', 'artık', 'asla', 'aslında', 'aslında', 'ayrıca', 'az', 'bana', 'bazen', 'bazı', 'bazıları', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beş', 'bile', 'bilhassa', 'bin', 'bir', 'biraz', 'birçoğu', 'birçok', 'biri', 'birisi', 'birkaç', 'birşey', 'biz', 'bizden', 'bize', 'bizi', 'bizim', 'böyle', 'böylece', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'burada', 'bütün', 'çoğu', 'çoğunu', 'çok', 'çünkü', 'da', 'daha', 'dahi', 'dan', 'de', 'defa', 'değil', 'diğer', 'diğeri'

In [5]:
def remove_stopwords(text):
  lst=[]
  for token in text.split():
    if token.lower() not in stops_tr:
      lst.append(token)
  return ' '.join(lst)

In [6]:
def tokens(words):
    words = re.sub("[^a-zA-ZğıüşöçİĞÜŞÖÇ]", " ", words)
    text = words.lower().split()
    return " ".join(text)

In [ ]:
from pandas import read_excel

my_sheet = 'Sayfa1' 
file_name = 'drive/My Drive/graduate/data/Canon_MD_classified.xlsx'
data = read_excel(file_name, sheet_name = my_sheet)

df = pd.DataFrame(data, columns =['id',	'from',	'subject',	'to',	'message',	'message_type',	'message_sync',	'message_id',	'date',	'special_message_type',	'is_read',	'supplier',	'order_number',	'conversation_id',	'url',	'productName',	'customer_id',	'productCode'])
df.info()

df.drop(columns=['from', 'id', 'to', 'message_type', 'message_sync','message_id', 'is_read', 'supplier', 'order_number','conversation_id', 'url', 'customer_id', 'date', 'special_message_type', 'productName', 'productCode'], inplace=True)

df.to_csv('drive/My Drive/graduate/data/Canon_MD_classified.csv')
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    748 non-null    float64       
 1   from                  743 non-null    object        
 2   subject               900 non-null    object        
 3   to                    285 non-null    object        
 4   message               900 non-null    object        
 5   message_type          83 non-null     object        
 6   message_sync          748 non-null    float64       
 7   message_id            748 non-null    object        
 8   date                  748 non-null    datetime64[ns]
 9   special_message_type  454 non-null    object        
 10  is_read               748 non-null    float64       
 11  supplier              748 non-null    object        
 12  order_number          60 non-null     float64       
 13  conversation_id     

In [ ]:
# df["productName"].value_counts()
# df["productCode"].value_counts()
# df["subject"].value_counts()

In [7]:
df_classified = pd.read_csv('drive/My Drive/graduate/data/Canon_MD_classified.csv')
df_classified['labels'] = pd.factorize(df_classified.subject)[0]
df_classified = df_classified.drop(df_classified.columns[0:2], axis=1)

# for bert classification model, column names and order is important. text area column must be "text", label must be "labels" respectively
df_classified.rename(columns={'message': 'text'}, inplace=True)
df_classified.text = [remove_stopwords(tokens(item)) for item in df_classified.text]

df_classified.head()

,text,labels
0,i̇nfluencerım video çekmek alacağım yeterli ür...,0
1,mm lens iyi lens,0
2,merhaba kutunun içinden renkli siyah beyaz ade...,1
3,kartuşu değiştirmeli dolum yapabiliyor muyuz,0
4,airprint özelliği varmı,0


In [8]:
# creating train and test dataset using train_test_split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_classified, test_size=0.2, random_state=42)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

#for bert text column should be string and label column should be int
train["text"]=train["text"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [10]:
# importing classification model, using simpletransformers lib which installed at https://colab.research.google.com/drive/1-W0AsgBT2-w0tqDY5l4FGt-veU_tuSYm#scrollTo=b8JeN8aNrI0w&line=1&uniqifier=1
from simpletransformers.classification import ClassificationModel

#create model with turkish bert
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=6, use_cuda=True,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":True, "output_dir": "drive/My Drive/graduate/model_nltk_2"})

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

In [11]:
#train model
model.train_model(train)

  0%|          | 0/720 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/12 [00:00<?, ?it/s]

(36, 1.332584063212077)

In [12]:
# testing model nd getting evaluation results
result, model_outputs, wrong_predictions = model.eval_model(test)

predictions = model_outputs.argmax(axis=1)
actuals = test.labels.values

  0%|          | 0/180 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

In [13]:
from sklearn import metrics

print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.621     0.986     0.762        73
           1      0.500     0.038     0.071        26
           2      0.800     0.148     0.250        27
           3      1.000     0.250     0.400        12
           4      0.704     0.927     0.800        41
           5      0.000     0.000     0.000         1

    accuracy                          0.656       180
   macro avg      0.604     0.392     0.381       180
weighted avg      0.671     0.656     0.566       180



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 30.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 362 kB 62.1 MB/s 
     |████████████████████████████████| 9.1 MB 74.3 MB/s 
     |████████████████████████████████| 1.8 MB 61.2 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 4.4 MB 62.6 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 181 kB 71.6 MB/s 
     |████████████████████████████████| 145 kB 71.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 212 kB 8.9 MB/s 
     |████████████████████████████████| 140 kB 75.2 MB/s 
     |██████████████████████████████

In [ ]:
!pip freeze > requirement.txt